In [4]:
import pandas as pd
import re

# Function to extract temperature and strain rate from sheet and column names
def extract_temp_strain_rate(sheet_name, column_name):
    # Extract temperature from the sheet name (e.g., 'Tmp77.15K_RD')
    temp_match = re.search(r'Tmp(\d+\.\d+K)', sheet_name)
    temp = float(temp_match.group(1)[:-1]) if temp_match else None
    
    # Extract strain rate from the column name (e.g., 'Tmp77.15K_StrRt0.0001_RD_E')
    strain_rate_match = re.search(r'StrRt(\d+\.\d+)', column_name)
    strain_rate = float(strain_rate_match.group(1)) if strain_rate_match else None
    
    return temp, strain_rate

# Load the Excel file
excel_path = 'Exp_FC_StrRtDpn_TmpDpn (1).xlsx'
excel_file = pd.ExcelFile(excel_path)

# Prepare a dataframe to hold the merged data
merged_data = pd.DataFrame(columns=["Strain", "Stress", "Strain Rate", "Temperature"])

# Iterate over each sheet in the Excel file
for sheet_name in excel_file.sheet_names:
    # Load the sheet
    sheet_data = excel_file.parse(sheet_name)
    
    # Extract strain and stress columns
    for column in sheet_data.columns:
        if column.endswith("_E"):  # Strain column
            strain_col = column
            stress_col = column.replace("_E", "_S")  # Find the matching stress column
            
            if stress_col in sheet_data.columns:
                # Extract the temperature and strain rate from the sheet and column names
                temp, strain_rate = extract_temp_strain_rate(sheet_name, column)
                
                temp = int(temp)
                
                # Prepare a temporary dataframe for this data
                temp_df = pd.DataFrame({
                    "Strain": sheet_data[strain_col],
                    "Stress": sheet_data[stress_col],
                    "Strain Rate": strain_rate,
                    "Temperature": temp     # Rounded Temp
                })
                
                # Append to the merged data
                merged_data = pd.concat([merged_data, temp_df], ignore_index=True)

# Remove rows with any NaN values
merged_data = merged_data.dropna()

# Round Strain and Stress to 5 decimal places
merged_data["Strain"] = merged_data["Strain"].round(5)
merged_data["Stress"] = merged_data["Stress"].round(7)

# Locate the reference stress (strain = 0.0002, strain rate = 0.0001, temperature = 298.15)
reference_condition = merged_data[
    (merged_data["Strain"] == 0.0002) & 
    (merged_data["Strain Rate"] == 0.0001) & 
    (merged_data["Temperature"] == 298)
]

if not reference_condition.empty:
    reference_stress = reference_condition.iloc[0]["Stress"]
else:
    reference_stress = None

# Normalize the stress column if reference stress is found
if reference_stress:
    merged_data["Normalized Stress"] = merged_data["Stress"] / reference_stress
    
# Save the merged data to a CSV file
output_csv_path = 'experiment_data_lstm.csv'
merged_data.to_csv(output_csv_path, index=False)

# Display the first few rows of the merged data
print(merged_data.head())

/tmp/ipykernel_291/2359561386.py:49: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged_data = pd.concat([merged_data, temp_df], ignore_index=True)


   Strain      Stress  Strain Rate Temperature  Normalized Stress
0  0.0000    0.000000       0.0001         298           0.000000
1  0.0002  580.282200       0.0001         298           1.000000
2  0.0004  644.631000       0.0001         298           1.110892
3  0.0006  681.693300       0.0001         298           1.174762
4  0.0008  706.112533       0.0001         298           1.216843
